# 07: Waveform Comparison

This notebook compares different waveform performance characteristics.

## Learning Objectives
- Compare spectral efficiency across modulations
- Analyze PAPR characteristics
- Understand tradeoffs between throughput and robustness
- Use R4W to evaluate waveforms

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import special
from r4w_python import plot_constellation, plot_spectrum, compute_spectrum, plot_ber_comparison

%matplotlib inline
plt.style.use('seaborn-v0_8-whitegrid')

## Spectral Efficiency

**Spectral efficiency** = bits/second/Hz = $\log_2(M)$ for M-ary modulation

| Modulation | M | Bits/Symbol | Spectral Efficiency |
|------------|---|-------------|--------------------|
| BPSK | 2 | 1 | 1 bit/s/Hz |
| QPSK | 4 | 2 | 2 bit/s/Hz |
| 8-PSK | 8 | 3 | 3 bit/s/Hz |
| 16-QAM | 16 | 4 | 4 bit/s/Hz |
| 64-QAM | 64 | 6 | 6 bit/s/Hz |

In [ ]:
# Generate constellations
def psk_constellation(m):
    return np.exp(1j * 2 * np.pi * np.arange(m) / m)

def qam_constellation(m):
    k = int(np.sqrt(m))
    points = []
    for i in range(k):
        for j in range(k):
            points.append(complex(2*i - (k-1), 2*j - (k-1)))
    return np.array(points) / np.sqrt(2*(k-1)**2/3)

constellations = {
    'BPSK': psk_constellation(2),
    'QPSK': psk_constellation(4) * np.exp(1j * np.pi/4),
    '8-PSK': psk_constellation(8),
    '16-QAM': qam_constellation(16),
    '64-QAM': qam_constellation(64),
}

In [ ]:
# Plot all constellations
fig, axes = plt.subplots(2, 3, figsize=(15, 10))
axes = axes.flatten()

for ax, (name, const) in zip(axes, constellations.items()):
    bits_per_sym = int(np.log2(len(const)))
    plot_constellation(const, title=f"{name} ({bits_per_sym} bit/sym)", 
                       ax=ax, marker_size=50 if len(const) < 32 else 15)

# Hide last subplot
axes[-1].axis('off')

plt.tight_layout()
plt.show()

## Minimum Distance

The **minimum distance** between constellation points determines noise tolerance.

$$d_{min} = \min_{i \neq j} |s_i - s_j|$$

In [ ]:
def min_distance(constellation):
    """Calculate minimum distance between constellation points."""
    d_min = np.inf
    for i, s1 in enumerate(constellation):
        for j, s2 in enumerate(constellation):
            if i != j:
                d = np.abs(s1 - s2)
                if d < d_min:
                    d_min = d
    return d_min

print("Minimum distance (normalized to unit average power):")
for name, const in constellations.items():
    # Normalize to unit power
    const_norm = const / np.sqrt(np.mean(np.abs(const)**2))
    d = min_distance(const_norm)
    print(f"  {name:10s}: d_min = {d:.3f}")

## BER Performance Comparison

Higher spectral efficiency requires higher SNR for the same BER.

In [ ]:
def q_function(x):
    return 0.5 * special.erfc(x / np.sqrt(2))

def ber_bpsk(eb_n0_db):
    eb_n0 = 10**(eb_n0_db / 10)
    return q_function(np.sqrt(2 * eb_n0))

def ber_qpsk(eb_n0_db):
    return ber_bpsk(eb_n0_db)  # Same as BPSK

def ber_8psk(eb_n0_db):
    eb_n0 = 10**(eb_n0_db / 10)
    return (2/3) * q_function(np.sqrt(2 * 3 * eb_n0) * np.sin(np.pi/8))

def ber_16qam(eb_n0_db):
    eb_n0 = 10**(eb_n0_db / 10)
    return (3/8) * special.erfc(np.sqrt((4/10) * eb_n0))

def ber_64qam(eb_n0_db):
    eb_n0 = 10**(eb_n0_db / 10)
    return (7/24) * special.erfc(np.sqrt((2/21) * eb_n0))

In [ ]:
# Plot BER comparison
eb_n0_db = np.linspace(0, 20, 200)

plt.figure(figsize=(10, 7))

ber_funcs = {
    'BPSK (1 bit/sym)': ber_bpsk,
    'QPSK (2 bit/sym)': ber_qpsk,
    '8-PSK (3 bit/sym)': ber_8psk,
    '16-QAM (4 bit/sym)': ber_16qam,
    '64-QAM (6 bit/sym)': ber_64qam,
}

for name, func in ber_funcs.items():
    plt.semilogy(eb_n0_db, func(eb_n0_db), label=name)

# Add target BER lines
plt.axhline(1e-3, color='gray', linestyle='--', alpha=0.5)
plt.text(0.5, 1.5e-3, 'Voice quality (10^-3)', fontsize=9)
plt.axhline(1e-6, color='gray', linestyle='--', alpha=0.5)
plt.text(0.5, 1.5e-6, 'Data quality (10^-6)', fontsize=9)

plt.xlabel('Eb/N0 (dB)')
plt.ylabel('Bit Error Rate')
plt.title('BER Comparison: Spectral Efficiency vs SNR Requirement')
plt.legend(loc='lower left')
plt.grid(True, which='both')
plt.ylim(1e-7, 1)
plt.show()

## Peak-to-Average Power Ratio (PAPR)

PAPR affects amplifier efficiency:

$$\text{PAPR} = \frac{\max|s(t)|^2}{E[|s(t)|^2]}$$

Higher PAPR requires more amplifier backoff.

In [ ]:
def calculate_papr_db(signal):
    """Calculate PAPR in dB."""
    peak_power = np.max(np.abs(signal)**2)
    avg_power = np.mean(np.abs(signal)**2)
    return 10 * np.log10(peak_power / avg_power)

# Simulate multi-carrier OFDM-like signals
def generate_ofdm_symbol(constellation, num_carriers=64):
    """Generate OFDM symbol with given constellation."""
    # Random data symbols
    data = constellation[np.random.randint(0, len(constellation), num_carriers)]
    # IFFT to get time domain
    return np.fft.ifft(data) * np.sqrt(num_carriers)

# Calculate PAPR for each modulation
np.random.seed(42)
num_trials = 1000

papr_results = {}
for name, const in constellations.items():
    paprs = [calculate_papr_db(generate_ofdm_symbol(const)) for _ in range(num_trials)]
    papr_results[name] = paprs
    print(f"{name:10s}: Mean PAPR = {np.mean(paprs):.2f} dB, 99th percentile = {np.percentile(paprs, 99):.2f} dB")

In [ ]:
# Plot PAPR CCDF
plt.figure(figsize=(10, 6))

for name, paprs in papr_results.items():
    sorted_papr = np.sort(paprs)
    ccdf = 1 - np.arange(1, len(sorted_papr)+1) / len(sorted_papr)
    plt.semilogy(sorted_papr, ccdf, label=name)

plt.xlabel('PAPR (dB)')
plt.ylabel('CCDF P(PAPR > x)')
plt.title('PAPR Complementary CDF (64-carrier OFDM)')
plt.legend()
plt.grid(True)
plt.xlim(4, 12)
plt.ylim(1e-3, 1)
plt.show()

## Summary: Waveform Selection Guidelines

| Scenario | Recommended | Reason |
|----------|-------------|--------|
| Low SNR, long range | BPSK/QPSK | Best noise tolerance |
| High throughput, good channel | 64-QAM | Maximum spectral efficiency |
| Nonlinear amplifier | FSK/GMSK | Constant envelope |
| Deep space | BPSK + FEC | Extreme sensitivity needed |
| Mobile wireless | Adaptive | Switch based on conditions |

In [ ]:
# Find SNR required for target BER
target_ber = 1e-5

print(f"Eb/N0 required for BER = {target_ber:.0e}:")
for name, func in ber_funcs.items():
    # Binary search for required SNR
    for snr in np.linspace(0, 30, 300):
        if func(snr) < target_ber:
            bits = int(name.split('(')[1].split()[0])
            print(f"  {name}: {snr:.1f} dB (Throughput @ 1 MHz = {bits:.1f} Mbps)")
            break

## Exercises

1. **Capacity comparison**: At 10 dB SNR, which modulation provides the highest throughput while maintaining BER < $10^{-4}$?

2. **PAPR reduction**: How would you reduce PAPR in OFDM systems?

3. **Adaptive modulation**: Design a table mapping SNR ranges to modulation schemes for a WiFi-like system.

In [ ]:
# Your code here
